In [0]:
%load_ext autoreload

from gensim.models import Word2Vec, KeyedVectors

## word2vec + fasttext

загрузка модели

In [0]:
# если модель без тэгов
model_file = '../data/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model'

# model = Word2Vec.load(model_path)

In [0]:
# если модель с POS-тэггингом
model_file = '../data/tayga.vec'

model = KeyedVectors.load_word2vec_format(model_file, binary=False)

проверка наличия слова в словаре

In [0]:
lemma = 'черепаха_NOUN'
lemma in model.wv

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


True

получение вектора слова

In [0]:
model.wv[lemma]

In [0]:
# model[lemma]

получение вектора документа

In [0]:
import numpy as np

# сделали препроцессинг, получили леммы 
lemmas = ['старинный_ADJ', 'замок_NOUN']

# создаем маски для векторов 
lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
vec = np.zeros((model.vector_size,))

# если слово есть в модели, берем его вектор
for idx, lemma in enumerate(lemmas):
    if lemma in model.wv:
        lemmas_vectors[idx] = model.wv[lemma]
        
# проверка на случай, если на вход пришел пустой массив
if lemmas_vectors.shape[0] is not 0:
    vec = np.mean(lemmas_vectors, axis=0)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  del sys.path[0]


## Задание

Реализуйте поиск по [Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian) на нескольких векторных моделях

    1. fasttext, модель ruscorpora_none_fasttextskipgram_300_2_2019
    2. elmo, модель ruwikiruscorpora_lemmas_elmo_1024_2019
    3. bert*, RuBERT - необязательно
   
Первые две обученные модели можно скачать на сайте [rusvectores](https://rusvectores.org/en/models/).

BERT делать необязательно, но если сделаете, 6 за курс у вас автоматом. Модель можно [найти тут](http://docs.deeppavlov.ai/en/master/features/models/bert.html).

<a link href="https://drive.google.com/file/d/1fidXZNcxusanYSmiY-4tq56_PKH0aKNk/view?usp=sharing"> Открыть в Google Colab </a>



<a link href="https://drive.google.com/open?id=1V6qKqm6ZjIONV3WIcSWpoJ8aa_jY7aXB"> Папка с данными на Google Drive </a>

#### Загрузка данных:

In [1]:
pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.1)


In [2]:
import gensim
gensim.__version__

'3.8.1'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
import os

In [5]:
os.listdir()

['.config', 'drive', 'sample_data']

In [0]:
os.chdir('drive/My Drive/infosearch_data')

In [8]:
os.listdir()

['models',
 'simple_elmo',
 'data',
 'str_example.npy',
 'float_example.npy',
 'semantics.ipynb',
 'ft_search_index.npy',
 'ft_search_texts.npy',
 'elmo_search_index.npy',
 'elmo_search_texts.npy',
 'semantics_final (1).ipynb']

In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
from scipy.sparse import csr_matrix

In [0]:
data = pd.read_csv("data/quora_question_pairs_rus.csv").dropna()

In [13]:
data.head()

,Unnamed: 0,question1,question2,is_duplicate
0,0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


In [0]:
## using hint from
## https://stackoverflow.com/questions/43203215/map-unique-strings-to-integers-in-python
def extract_texts(df):
    df = df.dropna()
    question1 = list(set(df['question1']))
    question2 = list(set(df['question2']))
    mapping1 = {j:i for i,j in enumerate(question1)}
    mapping2 = {j:i for i,j in enumerate(question2)}
    df = df[df['is_duplicate']==1]
    q1 = df['question1'].apply(lambda x: mapping1[x])
    q2 = df['question2'].apply(lambda x: mapping2[x])
    duplicate_matrix = csr_matrix((np.ones(df.shape[0]),
                                  (q1,q2)),
                                  shape=(len(question1),len(question2)))
    return question1, question2, mapping1, mapping2, duplicate_matrix

In [0]:
texts1, texts2, text1_to_id, text2_to_id, duplicate_matrix = extract_texts(data)

#### Инициализация парсера:

In [16]:
pip install pymorphy2

In [0]:
from pymorphy2.tokenizers import simple_word_tokenize as tokenize

In [0]:
from pymorphy2.analyzer import MorphAnalyzer

In [0]:
analyzer = MorphAnalyzer()

In [0]:
def lemmatize(text):
    return [parse.normal_form for parse in [analyzer.parse(word)[0] for word in tokenize(text)] if parse.tag.POS]

In [21]:
lemmatize('Мир замер в ожидании страшных вещей - ты слышишь голос овощей!')

['мир',
 'замереть',
 'в',
 'ожидание',
 'страшный',
 'вещий',
 'ты',
 'слышать',
 'голос',
 'овощ']

### __Задача 1__:    
Сравните время индексации корпуса для каждой модели 

Инициализируем модели:

In [0]:
from gensim.models import KeyedVectors

Fast Text:

In [0]:
ft_model = KeyedVectors.load('models/fasttext/model.model')

In [24]:
ft_model.vector_size

300

Elmo:

In [0]:
import sys

In [0]:
import time

In [0]:
import tensorflow as tf
sys.path.append('simple_elmo')
from elmo_helpers import load_elmo_embeddings, get_elmo_vectors

elmo_path = 'models/elmo'

In [0]:
from sklearn.preprocessing import normalize

Определим классы для поиска:

Сначала определим базовый класс:




In [0]:
class IndexSearch:
    def search(self, text, return_indices=False, n_results=5):
        query = normalize(self.transform(text)[np.newaxis])
        sims = np.dot(query, self.index)
        top_texts = np.argsort(sims[0]).tolist()[::-1]
        top_texts = top_texts[:n_results]
        if return_indices:
            return top_texts
        else:
            return self.texts[top_texts]
   
    def dump(self, index_file, text_collection_file):
        self.index.dump(index_file)
        self.texts.dump(text_collection_file)

Теперь определим класс для поиска с использованием модели Word2Vec/FastText:

In [0]:
class W2VSearch(IndexSearch):
    def __init__(self, model, texts, lemmatizer=lemmatize,
                use_pretrained=False, index_file=None):
        self.model = model
        self.lemmatize = lemmatizer
        if use_pretrained:
            if index_file:
                self.index = np.load(index_file, allow_pickle=True)
                if type(texts) == str:
                    self.texts = np.load(texts, allow_pickle=True)
                else:
                    self.texts = np.array(texts)
            else:
                raise ValueError("No index file provided")
        else:
            self.texts = np.array(texts)
            self.index = np.zeros(shape=(len(texts),model.vector_size))
            for text_id, text in enumerate(texts):
                if text:
                    self.index[text_id] = self.transform(text)
            self.texts = self.texts[~np.isnan(self.index).any(axis=1)]
            self.index = self.index[~np.isnan(self.index).any(axis=1)]
            self.index = normalize(self.index)
            self.index = self.index.transpose() 
    
    def transform(self, text):
        vecs = [self.model[word] for word in self.lemmatize(text) if word in self.model]
        return np.mean(vecs, axis=0)

In [0]:
%%time
global search
search = W2VSearch(ft_model, texts2)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


CPU times: user 9min 57s, sys: 3.49 s, total: 10min 1s
Wall time: 10min 1s


In [0]:
search.search('кто такие фиксики')

array(['кто такие дети', 'кто такой jose mourinho',
       'кто такой harsha bhogle', 'кто такой karl marx',
       'кто такой шив-надар'], dtype='<U1283')

In [0]:
search.search('рецепт макарон по-клингонски')

array(['лучший рецепт японского жареного риса', 'лучший рецепт сэндвича',
       'лучший рецепт печеного картофеля',
       'какие хорошие приправы добавить к рецепту картофельного пюре',
       'какие лучшие заменители лука-порея в рецепте супа'],
      dtype='<U1283')

In [0]:
search.dump("ft_search_index.npy", "ft_search_texts.npy")

In [0]:
del search, ft_model, gensim, Word2Vec, KeyedVectors

Определим класс для поиска по модели elmo:

In [0]:
class ElmoSearch(IndexSearch):
    def __init__(self, texts, elmo_path, batch_size=20, lemmatizer=lemmatize,
                use_pretrained=False, index_file=None):
        self.elmo_path = elmo_path
        self.lemmatize = lemmatizer
        tf.reset_default_graph()
        self.batcher, self.scids, self.esi = load_elmo_embeddings(elmo_path)
        if use_pretrained:
            if index_file:
                self.index = np.load(index_file, allow_pickle=True)
                if type(texts) == str:
                    self.texts = np.load(texts, allow_pickle=True)
                else:
                    self.texts = np.array(texts)
            else:
                raise ValueError("No index file provided")
        else:
            texts = sorted(texts, key=len, reverse=True)
            self.index = self.multiple_transform(texts, batch_size=batch_size)
            self.texts = np.array(texts)
            self.texts = self.texts[~np.isnan(self.index).any(axis=1)]
            self.index = self.index[~np.isnan(self.index).any(axis=1)]
            self.index = normalize(self.index)
            self.index = self.index.transpose()
    
    def multiple_transform(self, texts, batch_size=200):
        index = np.zeros(shape=(len(texts), self.esi['weighted_op'].shape[2].value))
        for i in range(0, len(texts), batch_size):
            j = min(i+batch_size, len(texts))
            text_batch = [self.lemmatize(text) for text in texts[i:j]]
            print("max sentence length - ", max([len(i) for i in text_batch]))
            with tf.Session() as sess:
                sess.run(tf.global_variables_initializer())
                elmo_vectors = get_elmo_vectors(
                sess, text_batch, self.batcher, self.scids, self.esi)
            index[i:j] = np.mean(elmo_vectors, axis=1)
            del elmo_vectors
        return index
   
    def transform(self, text):
        return self.multiple_transform([text])[0]

In [0]:
import time
t = time.time()
search_elmo = ElmoSearch(texts2, elmo_path, batch_size=100)
print(time.time() - t)

max sentence length -  214


Sentences in this batch: 100


max sentence length -  62


Sentences in this batch: 100


max sentence length -  63


...

max sentence length -  2


Sentences in this batch: 100


max sentence length -  1


Sentences in this batch: 7


7926.146076440811


Поисковая модель с ELMO обучилась чуть более чем за 2 часа с использованием GPU.

In [0]:
ElmoSearch.search(search_elmo, "кто такие фиксики")

max sentence length -  3


Sentences in this batch: 1


array(['кто такой шив-надар', 'кто такой харуки мураками',
       'кто такой чак норрис', 'кто такой амитаб баччан',
       'кто такой джина-карано'], dtype='<U1283')

In [0]:
ElmoSearch.search(search_elmo, "рецепт макарон по-клингонски")

max sentence length -  3


Sentences in this batch: 1


array(['лучший рецепт сыра на гриле', 'лучший рецепт сэндвича',
       'что такое альтернативный рецепт салата из немецкого картофеля без бекона',
       'лучший рецепт приготовления хрустящей кукурузы',
       'лучший рецепт японского жареного риса'], dtype='<U1283')

Сохраним полученную модель:

In [0]:
ElmoSearch.dump(search_elmo, "elmo_search_index.npy", "elmo_search_texts.npy")

In [0]:
del search_elmo

### __Задача 2__:    
Выведите качество поиска для каждой модели +  BM25 для сравнения

Качество оцениваем так же, как в прошлом задании:
    - если в топ-5 результатов выдачи попал хоть один релевантный документ, выдача точная
    - если в топ-5 нет ни одного релеватного документа, выдача получает 0
   

In [32]:
elmo_search1 = ElmoSearch(elmo_path=elmo_path, use_pretrained=True,
                          texts = "elmo_search_texts.npy",
                          index_file = "elmo_search_index.npy")





Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






In [0]:
ft_search1 = W2VSearch(
                       use_pretrained = True,
                       model = KeyedVectors.load('models/fasttext/model.model'),
                       texts = "ft_search_texts.npy",
                       index_file = "ft_search_index.npy"
                      )

In [36]:
elmo_search1.search("Усама бин Ладен")

max sentence length -  3


Sentences in this batch: 1


array(['Осама бен Ладен все еще жив', 'как осама бин ладен умирает',
       'кто был осама бин ладен', 'салман хан гей',
       'как убил осама бен ладен'], dtype='<U1283')

In [37]:
ft_search1.search("Усама бин Ладен")

array(['кто был осама бин ладен', 'как осама бин ладен умирает',
       'то, что пытался сделать осама бин ладен',
       'как были груженый садсин и осама бен ладен связаны',
       'как убил осама бен ладен'], dtype='<U1283')

In [0]:
import random

In [0]:
def evaluate(search_engine, texts, text2id, duplicate_matrix, test_size=10000):
    s = 0
    N = 0
    
    random.seed(42)
    texts = random.choices(list(enumerate(texts)), k=test_size)
    
    for text_id, text in texts:
        if duplicate_matrix[text_id,].sum():
            N += 1
            results = search_engine.search(text)
            for result in results:
                result_id = text2id[result]
                if duplicate_matrix[text_id, result_id]:
                    s += 1
                    break
    return s/N    

In [40]:
%%time
evaluate(elmo_search1, texts1, text2_to_id, duplicate_matrix, test_size=100)

max sentence length -  8


Sentences in this batch: 1


max sentence length -  7


Sentences in this batch: 1


max sentence length -  3


...

Sentences in this batch: 1


max sentence length -  13


Sentences in this batch: 1


CPU times: user 1min 8s, sys: 28 s, total: 1min 36s
Wall time: 1min 17s


0.5925925925925926

In [41]:
%%time
evaluate(ft_search1, texts1, text2_to_id, duplicate_matrix, test_size=100)

CPU times: user 7.09 s, sys: 6.92 s, total: 14 s
Wall time: 3.64 s


0.5185185185185185

Лучший результат на основе 100 случайных текстов показывает поиск на основе моделb Elmo (0.59 vs. 0.52). Для сравнения также посчитаем качество поиска для BM25:

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
class BMVectorSearch:
    def __init__(self, docs, lemmatizer=lemmatize, k=2, b=0.75):
        self.k = k
        self.b = b
        self.docs = []
        self.lemmatizer = lemmatizer
        self.update_index(docs)
    
    def update_index(self, docs):
        self.docs = np.array(list(self.docs) + docs)
        self.vectorizer = CountVectorizer(tokenizer=self.lemmatizer)
        self.tf_matrix = self.vectorizer.fit_transform(self.docs)
        ## counting non-zero values in sparse matrix row
        ## using only sparse matrix operations:
        ## dense matrix would be too big - 300+ GB (vs. ~500MB in sparse format)
        df = self.tf_matrix.getnnz(axis=1)
        N = df.sum()
        idf_transform = np.vectorize(lambda x: (N-x+0.5)/(x+0.5))
        self.idf = np.log(idf_transform(df))
        self.set_hyperparams()
    
    def set_hyperparams(self, k=None, b=None):
        if k is None:
            k = self.k
        else:
            self.k = k
        if b is None:
            b = self.b
        else:
            self.b = b
        doc_lengths = np.array(self.tf_matrix.sum(axis=0))[0]
        avgdl = doc_lengths.mean()
        length_ratios = np.array([doc_lengths[col]/avgdl for col in self.tf_matrix.nonzero()[1]])
        idfs = np.array([self.idf[row] for row in self.tf_matrix.nonzero()[0]])
        new_data = idfs * self.tf_matrix.data * (k+1) / (self.tf_matrix.data + k*(1-b+b*length_ratios))
        self.index = csr_matrix((new_data, self.tf_matrix.indices, self.tf_matrix.indptr),
                                shape=self.tf_matrix.shape)
        self.index = self.index.transpose(copy=True)
    
    def search(self, query, return_similarities=False, return_indices=False, n_results=5):
        vector = self.vectorizer.transform([query])
        similarities = vector * self.index
        similarities = np.array(similarities.todense())[0]
        order = np.argsort(similarities)[::-1].tolist()[:n_results]
        
        if return_similarities:
            if return_indices:
                return similarities[order], order
            return similarities[order], self.docs[order]
        
        if return_indices:
            return order
        return self.docs[order]
    
    def multiple_search(self, query):
        vector = self.vectorizer.transform(query)
        return vector * self.index

In [45]:
%%time
global search
search = BMVectorSearch(texts2)

CPU times: user 10min 33s, sys: 1.24 s, total: 10min 34s
Wall time: 10min 34s


In [46]:
evaluate(search, texts1, text2_to_id, duplicate_matrix, test_size=100)

0.37037037037037035

Заметим, что BM25 показывает гораздо худший результат (0.37)